In [4]:
from PyQt5 import QtWidgets, QtGui, QtCore
import sys
import os

class SplashScreen(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        self.setAttribute(QtCore.Qt.WA_TranslucentBackground)
        self.initUI()
        
    def initUI(self):
        layout = QtWidgets.QVBoxLayout()
        label = QtWidgets.QLabel()
        pixmap = QtGui.QPixmap("AMS_Logo_Final_Removed.png")
        label.setPixmap(pixmap.scaledToWidth(self.width()))
        layout.addWidget(label)
        self.setLayout(layout)
        QtCore.QTimer.singleShot(4000, self.show_main_window)
    
    def show_main_window(self):
        self.main_window = MainWindow()
        self.main_window.show()
        self.close()

class MainWindow(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle("Prompt and Confidence Tuning")
        self.showFullScreen()
        self.setStyleSheet("background-color: black;")
        
        layout = QtWidgets.QVBoxLayout()
        
        exit_btn = QtWidgets.QPushButton("Exit")
        exit_btn.setStyleSheet("background-color: red; color: white; font-size: 24px;")
        exit_btn.setFixedSize(200, 100)
        exit_btn.clicked.connect(self.close)
        layout.addWidget(exit_btn, alignment=QtCore.Qt.AlignTop | QtCore.Qt.AlignLeft)
        
        button_layout = QtWidgets.QVBoxLayout()
        
        manual_btn = QtWidgets.QPushButton("Manual Prompt and Confidence Tuning")
        manual_btn.setStyleSheet("background-color: blue; color: white; font-size: 36px;")
        manual_btn.setFixedSize(800, 150)  # Adjusted width, same height
        manual_btn.clicked.connect(self.select_manual)
        button_layout.addWidget(manual_btn, alignment=QtCore.Qt.AlignCenter)
        
        automated_btn = QtWidgets.QPushButton("Automated Prompt and Confidence Tuning")
        automated_btn.setStyleSheet("background-color: red; color: white; font-size: 36px;")
        automated_btn.setFixedSize(800, 150)  # Adjusted width, same height
        automated_btn.clicked.connect(self.select_automated)
        button_layout.addWidget(automated_btn, alignment=QtCore.Qt.AlignCenter)
        
        layout.addStretch()
        layout.addLayout(button_layout)
        layout.addStretch()
        layout.setAlignment(button_layout, QtCore.Qt.AlignCenter)
        self.setLayout(layout)
    
    def select_manual(self):
        self.manual_window = ManualWindow()
        self.manual_window.show()
        self.hide()
    
    def select_automated(self):
        self.automated_window = AutomatedWindow()
        self.automated_window.show()
        self.hide()
        
class ManualWindow(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle("Manual Prompt and Confidence Tuning")
        self.showFullScreen()
        self.setStyleSheet("background-color: black;")
        
        layout = QtWidgets.QVBoxLayout()
        
        back_btn = QtWidgets.QPushButton("Back")
        back_btn.setStyleSheet("background-color: grey; color: white; font-size: 24px;")
        back_btn.setFixedSize(200, 100)
        back_btn.clicked.connect(self.go_back)
        layout.addWidget(back_btn, alignment=QtCore.Qt.AlignTop | QtCore.Qt.AlignLeft)
        
        folder_btn = QtWidgets.QPushButton("Select Image Folder")
        folder_btn.setStyleSheet("background-color: blue; color: white; font-size: 24px;")
        folder_btn.setFixedSize(400, 100)
        folder_btn.clicked.connect(self.select_folder)
        layout.addWidget(folder_btn, alignment=QtCore.Qt.AlignCenter)
        
        self.image_label = QtWidgets.QLabel()
        layout.addWidget(self.image_label, alignment=QtCore.Qt.AlignRight)
        
        prompt_label = QtWidgets.QLabel("Enter Prompt:")
        prompt_label.setStyleSheet("color: white; font-size: 24px;")
        layout.addWidget(prompt_label, alignment=QtCore.Qt.AlignLeft)
        
        self.prompt_entry = QtWidgets.QLineEdit()
        self.prompt_entry.setStyleSheet("font-size: 24px; color: white; background-color: black;")
        self.prompt_entry.setFixedHeight(50)
        self.prompt_entry.setFixedWidth(800)  # Double the width
        layout.addWidget(self.prompt_entry, alignment=QtCore.Qt.AlignLeft)
        
        spacer1 = QtWidgets.QSpacerItem(20, 50, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding)
        layout.addItem(spacer1)
        
        confidence_label = QtWidgets.QLabel("Confidence: 50")
        confidence_label.setStyleSheet("color: white; font-size: 24px;")
        layout.addWidget(confidence_label, alignment=QtCore.Qt.AlignLeft)
        self.confidence_label = confidence_label  # Save reference to update later
        
        self.confidence_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.confidence_slider.setRange(0, 100)
        self.confidence_slider.setValue(50)
        self.confidence_slider.setStyleSheet("font-size: 24px;")
        self.confidence_slider.setFixedSize(800, 50)  # Double the width
        layout.addWidget(self.confidence_slider, alignment=QtCore.Qt.AlignLeft)
        self.confidence_slider.valueChanged.connect(self.update_confidence_value)
        
        spacer2 = QtWidgets.QSpacerItem(20, 25, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding)
        layout.addItem(spacer2)
        
        box_threshold_label = QtWidgets.QLabel("Box Threshold: 90")
        box_threshold_label.setStyleSheet("color: white; font-size: 24px;")
        layout.addWidget(box_threshold_label, alignment=QtCore.Qt.AlignLeft)
        self.box_threshold_label = box_threshold_label  # Save reference to update later
        
        self.box_threshold_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.box_threshold_slider.setRange(0, 100)
        self.box_threshold_slider.setValue(90)
        self.box_threshold_slider.setStyleSheet("font-size: 24px;")
        self.box_threshold_slider.setFixedSize(800, 50)  # Double the width
        layout.addWidget(self.box_threshold_slider, alignment=QtCore.Qt.AlignLeft)
        self.box_threshold_slider.valueChanged.connect(self.update_box_threshold_value)
        
        spacer3 = QtWidgets.QSpacerItem(20, 200, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding)
        layout.addItem(spacer3)
        
        bottom_layout = QtWidgets.QHBoxLayout()
        
        next_btn = QtWidgets.QPushButton("Next Image")
        next_btn.setStyleSheet("background-color: green; color: white; font-size: 24px;")
        next_btn.setFixedSize(400, 100)
        next_btn.clicked.connect(self.next_image)
        bottom_layout.addWidget(next_btn, alignment=QtCore.Qt.AlignBottom | QtCore.Qt.AlignLeft)
        
        auto_annotate_btn = QtWidgets.QPushButton("Auto Annotate Remaining")
        auto_annotate_btn.setStyleSheet("background-color: red; color: white; font-size: 24px;")
        auto_annotate_btn.setFixedSize(400, 100)
        auto_annotate_btn.clicked.connect(self.auto_annotate_remaining)
        bottom_layout.addWidget(auto_annotate_btn, alignment=QtCore.Qt.AlignBottom | QtCore.Qt.AlignRight)
        
        layout.addLayout(bottom_layout)
        self.setLayout(layout)
        
        self.current_image_index = 0
        self.images = []
    
    def go_back(self):
        self.main_window = MainWindow()
        self.main_window.show()
        self.close()
    
    def select_folder(self):
        options = QtWidgets.QFileDialog.Options()
        options |= QtWidgets.QFileDialog.DontUseNativeDialog
        options |= QtWidgets.QFileDialog.ReadOnly
        dialog = QtWidgets.QFileDialog(self, "Select Image Folder", options=options)
        dialog.setFileMode(QtWidgets.QFileDialog.Directory)
        dialog.setOption(QtWidgets.QFileDialog.ShowDirsOnly, True)
        dialog.setStyleSheet("QWidget { background-color: white; color: black; }")
        if dialog.exec_() == QtWidgets.QDialog.Accepted:
            folder = dialog.selectedFiles()[0]
            if folder:
                self.images = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
                if self.images:
                    self.display_image(self.images[0])
                    self.current_image_index = 0
                else:
                    message_box = QtWidgets.QMessageBox()
                    message_box.setStyleSheet("QLabel { color: black; font-size: 24px; } QMessageBox { background-color: white; }")
                    message_box.setText("The selected folder does not contain any images.")
                    message_box.exec_()
    
    def display_image(self, image_path):
        pixmap = QtGui.QPixmap(image_path)
        self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), QtCore.Qt.KeepAspectRatio))
    
    def next_image(self):
        self.current_image_index += 1
        if self.current_image_index < len(self.images):
            self.display_image(self.images[self.current_image_index])
        else:
            message_box = QtWidgets.QMessageBox()
            message_box.setStyleSheet("QLabel { color: black; font-size: 24px; } QMessageBox { background-color: white; }")
            message_box.setText("No more images in the folder.")
            message_box.exec_()
    
    def auto_annotate_remaining(self):
        options = QtWidgets.QFileDialog.Options()
        options |= QtWidgets.QFileDialog.DontUseNativeDialog
        options |= QtWidgets.QFileDialog.ReadOnly
        dialog = QtWidgets.QFileDialog(self, "Select Output Folder", options=options)
        dialog.setFileMode(QtWidgets.QFileDialog.Directory)
        dialog.setOption(QtWidgets.QFileDialog.ShowDirsOnly, True)
        dialog.setStyleSheet("QWidget { background-color: white; color: black; }")
        if dialog.exec_() == QtWidgets.QDialog.Accepted:
            output_folder = dialog.selectedFiles()[0]
            if output_folder:
                # Add your auto-annotation logic here
                message_box = QtWidgets.QMessageBox()
                message_box.setStyleSheet("QLabel { color: black; font-size: 24px; } QMessageBox { background-color: white; }")
                message_box.setText("Annotations saved to the output folder.")
                message_box.exec_()
    
    def update_confidence_value(self, value):
        self.confidence_label.setText(f"Confidence: {value}")

    def update_box_threshold_value(self, value):
        self.box_threshold_label.setText(f"Box Threshold: {value}")



class AutomatedWindow(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle("Automated Prompt and Confidence Tuning")
        self.showFullScreen()
        self.setStyleSheet("background-color: black;")
        
        layout = QtWidgets.QVBoxLayout()
        
        back_btn = QtWidgets.QPushButton("Back")
        back_btn.setStyleSheet("background-color: grey; color: white; font-size: 24px;")
        back_btn.setFixedSize(200, 100)
        back_btn.clicked.connect(self.go_back)
        layout.addWidget(back_btn, alignment=QtCore.Qt.AlignTop | QtCore.Qt.AlignLeft)
        
        # Left layout for folder selection
        self.left_layout = QtWidgets.QVBoxLayout()
        
        label_btn = QtWidgets.QPushButton("Select Labelled Images")
        label_btn.setStyleSheet("background-color: blue; color: white; font-size: 24px;")
        label_btn.setFixedSize(400, 100)
        label_btn.clicked.connect(self.select_labelled_images)
        self.left_layout.addWidget(label_btn, alignment=QtCore.Qt.AlignTop)
        
        self.labelled_folder_label = QtWidgets.QLabel("")
        self.labelled_folder_label.setStyleSheet("color: white; font-size: 24px;")
        self.left_layout.addWidget(self.labelled_folder_label, alignment=QtCore.Qt.AlignTop)
        
        unannotated_btn = QtWidgets.QPushButton("Select Unannotated Image Folder")
        unannotated_btn.setStyleSheet("background-color: blue; color: white; font-size: 24px;")
        unannotated_btn.setFixedSize(400, 100)
        unannotated_btn.clicked.connect(self.select_unannotated_folder)
        self.left_layout.addWidget(unannotated_btn, alignment=QtCore.Qt.AlignTop)
        
        self.unannotated_folder_label = QtWidgets.QLabel("")
        self.unannotated_folder_label.setStyleSheet("color: white; font-size: 24px;")
        self.left_layout.addWidget(self.unannotated_folder_label, alignment=QtCore.Qt.AlignTop)
        
        output_btn = QtWidgets.QPushButton("Select Output Folder")
        output_btn.setStyleSheet("background-color: blue; color: white; font-size: 24px;")
        output_btn.setFixedSize(400, 100)
        output_btn.clicked.connect(self.select_output_folder)
        self.left_layout.addWidget(output_btn, alignment=QtCore.Qt.AlignTop)
        
        self.output_folder_label = QtWidgets.QLabel("")
        self.output_folder_label.setStyleSheet("color: white; font-size: 24px;")
        self.left_layout.addWidget(self.output_folder_label, alignment=QtCore.Qt.AlignTop)
        
        # Right layout for prompt selection
        self.right_layout = QtWidgets.QVBoxLayout()
        
        prompt_select_btn = QtWidgets.QPushButton("Prompt Selection")
        prompt_select_btn.setStyleSheet("background-color: blue; color: white; font-size: 24px;")
        prompt_select_btn.setFixedSize(400, 100)
        prompt_select_btn.clicked.connect(self.prompt_selection)
        self.right_layout.addWidget(prompt_select_btn, alignment=QtCore.Qt.AlignTop)
        
        bottom_layout = QtWidgets.QVBoxLayout()
        
        start_btn = QtWidgets.QPushButton("Start Annotation")
        start_btn.setStyleSheet("background-color: green; color: white; font-size: 24px;")
        start_btn.setFixedSize(400, 100)
        start_btn.clicked.connect(self.perform_automatic_annotation)
        bottom_layout.addWidget(start_btn, alignment=QtCore.Qt.AlignCenter)
        
        main_layout = QtWidgets.QHBoxLayout()
        main_layout.addLayout(self.left_layout)
        main_layout.addLayout(self.right_layout)
        
        layout.addLayout(main_layout)
        layout.addLayout(bottom_layout)
        self.setLayout(layout)
        
    def go_back(self):
        self.main_window = MainWindow()
        self.main_window.show()
        self.close()
    
    def select_labelled_images(self):
        options = QtWidgets.QFileDialog.Options()
        options |= QtWidgets.QFileDialog.DontUseNativeDialog
        options |= QtWidgets.QFileDialog.ReadOnly
        dialog = QtWidgets.QFileDialog(self, "Select Labelled Image Folder", options=options)
        dialog.setFileMode(QtWidgets.QFileDialog.Directory)
        dialog.setOption(QtWidgets.QFileDialog.ShowDirsOnly, True)
        dialog.setStyleSheet("QWidget { background-color: white; color: black; }")
        if dialog.exec_() == QtWidgets.QDialog.Accepted:
            labelled_folder = dialog.selectedFiles()[0]
            if labelled_folder:
                self.labelled_folder_label.setText(f"Labelled Folder: {labelled_folder}")
        
    def select_unannotated_folder(self):
        options = QtWidgets.QFileDialog.Options()
        options |= QtWidgets.QFileDialog.DontUseNativeDialog
        dialog = QtWidgets.QFileDialog(self, "Select Unannotated Image Folder", options=options)
        dialog.setFileMode(QtWidgets.QFileDialog.Directory)
        dialog.setOption(QtWidgets.QFileDialog.ShowDirsOnly, True)
        dialog.setStyleSheet("QWidget { background-color: white; color: black; }")
        if dialog.exec_() == QtWidgets.QDialog.Accepted:
            unannotated_folder = dialog.selectedFiles()[0]
            if unannotated_folder:
                self.unannotated_folder_label.setText(f"Unannotated Folder: {unannotated_folder}")
    
    def select_output_folder(self):
        options = QtWidgets.QFileDialog.Options()
        options |= QtWidgets.QFileDialog.DontUseNativeDialog
        dialog = QtWidgets.QFileDialog(self, "Select Output Folder", options=options)
        dialog.setFileMode(QtWidgets.QFileDialog.Directory)
        dialog.setOption(QtWidgets.QFileDialog.ShowDirsOnly, True)
        dialog.setStyleSheet("QWidget { background-color: white; color: black; }")
        if dialog.exec_() == QtWidgets.QDialog.Accepted:
            output_folder = dialog.selectedFiles()[0]
            if output_folder:
                self.output_folder_label.setText(f"Output Folder: {output_folder}")
    
    def perform_automatic_annotation(self):
        message_box = QtWidgets.QMessageBox()
        message_box.setStyleSheet("QLabel { color: black; font-size: 24px; } QMessageBox { background-color: white; }")
        message_box.setText("Annotations saved to the output folder.")
        message_box.exec_()
    
    def prompt_selection(self):
        if not hasattr(self, "prompt_buttons_added"):
            list_prompts_btn = QtWidgets.QPushButton("List of Prompts")
            list_prompts_btn.setStyleSheet("background-color: blue; color: white; font-size: 24px;")
            list_prompts_btn.setFixedSize(400, 100)
            list_prompts_btn.clicked.connect(self.handle_list_of_prompts)
            self.right_layout.addWidget(list_prompts_btn, alignment=QtCore.Qt.AlignTop)
        
            generate_prompts_btn = QtWidgets.QPushButton("Generate Prompts")
            generate_prompts_btn.setStyleSheet("background-color: blue; color: white; font-size: 24px;")
            generate_prompts_btn.setFixedSize(400, 100)
            generate_prompts_btn.clicked.connect(self.handle_generate_prompts)
            self.right_layout.addWidget(generate_prompts_btn, alignment=QtCore.Qt.AlignTop)
            
            self.prompt_buttons_added = True
    
    def handle_list_of_prompts(self):
        message_box = QtWidgets.QMessageBox()
        message_box.setStyleSheet("QLabel { color: black; font-size: 24px; } QMessageBox { background-color: white; }")
        message_box.setText("Selected list of prompts.")
        message_box.exec_()
        
    def handle_generate_prompts(self):
        options = QtWidgets.QFileDialog.Options()
        options |= QtWidgets.QFileDialog.DontUseNativeDialog
        dialog = QtWidgets.QFileDialog(self, "Select Sample Image", "", "Image Files (*.png *.jpg *.jpeg)", options=options)
        dialog.setStyleSheet("QWidget { background-color: white; color: black; }")
        if dialog.exec_() == QtWidgets.QDialog.Accepted:
            sample_image_path = dialog.selectedFiles()[0]
            if sample_image_path:
                prompt = "a sample prompt"  # Update this as needed
                prompts = generate_prompts(sample_image_path, prompt)
                if prompts:
                    message_box = QtWidgets.QMessageBox()
                    message_box.setStyleSheet("QLabel { color: black; font-size: 24px; } QMessageBox { background-color: white; }")
                    message_box.setText("\n".join(prompts))
                    message_box.exec_()

def generate_prompts(image_path, prompt):
    # This is a placeholder for the actual logic to generate prompts
    return ["Prompt 1", "Prompt 2", "Prompt 3"]
    

def main():
    app = QtWidgets.QApplication(sys.argv)
    splash = SplashScreen()
    splash.show()
    sys.exit(app.exec_())

ModuleNotFoundError: No module named 'groundingdino'

In [6]:
main()

SystemExit: 0

C:\Users\Mechanized Systems\.conda\envs\autoannotate\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
